In [1]:
import copy

import torch
import torch.nn as nn
#import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, AutoModelForMaskedLM, BertTokenizer, BertModel, BertConfig
from peft import LoraConfig, get_peft_model
from tqdm import tqdm

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
#AutoConfig.from_pretrained("bert-base-uncased")

In [4]:
#BertConfig.from_pretrained("bert-base-uncased")
#config[]
#BertModel.from_pretrained("bert-base-uncased", max_position_embeddings = 64,ignore_mismatched_sizes=True)

In [5]:
#model.cls.predictions.decoder

In [6]:
#bert_model = BertModel.from_pretrained("bert-base-uncased", max_position_embeddings = 64,ignore_mismatched_sizes=True)
import copy
bert_model = BertModel.from_pretrained("bert-base-uncased",output_hidden_states=True,return_dict=True)
#bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
#bert_model = BertModel.from_pretrained("bert-base-uncased", max_position_embeddings = 64,ignore_mismatched_sizes=True)
ori_model = copy.deepcopy(bert_model)
for param in bert_model.parameters():
    param.requires_grad = False
frozen_model = copy.deepcopy(bert_model)
from peft import LoraConfig, get_peft_model, PrefixTuningConfig
lora_config = LoraConfig(
r=8, #low rank
lora_alpha=32, #alpha scaling， scale lora weights/outputs
# target_modules=["q_proj", "v_proj"], #if you know the
lora_dropout=0.1,
#target_modules= ["bert"],
task_type="MLM" # set this for CLM or Seq2Seq
)
#ptuning_config = PrefixTuningConfig(task_type = "SEQ_CLS", num_virtual_tokens=10)
lora_model = get_peft_model(bert_model, lora_config)
#lora_model
#ptuning_model = get_peft_model(bert_model, ptuning_config)

In [7]:
class RegBert(nn.Module):
    def __init__(self):
        super(RegBert, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)
        #self.bert = model
        #self.config = BertConfig()
        #self.bert = BertModel.from_pretrained("bert-base-uncased", max_position_embeddings = 512,ignore_mismatched_sizes=True)
        #self.bert = BertModel.from_pretrained("bert-base-uncased")
        #for param in self.bert.parameters():
        #    param.requires_grad = False
        #self.bert.cls.predictions.decoder = nn.Linear(in_features=768, out_features= 200, bias=True)
        #self.fc1 = nn.Linear(768, 200)
        self.model = frozen_model
        #self.model = bert_model
        #self.linear = nn.Linear(768*3, 2)
        self.linear = nn.Linear(768, 2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        input_ids, attention_mask = x[0].to(self.device), x[1].to(self.device)
        hidden_out = self.model(input_ids, attention_mask=attention_mask, output_hidden_states=False)
        # Use specific number of layers of hidden states of "CLS" token
        #predict = self.linear(torch.concat(hidden_out.hidden_states[-3][:,0,:],-1))
        #Use the last pooler output of "CLS" token, more commom in fine-tuning
        predict = self.linear(hidden_out.pooler_output)
        #predict = self.linear(hidden_out.hidden_states[-1])
        return self.softmax(predict)
        #return self.softmax(hidden_out.logits)

In [8]:
#ori_model

In [9]:
SA_model = RegBert()
SA_model.to(device)

cuda


RegBert(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [10]:
def print_trainable_parameters(to_model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in to_model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
            print(param)
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(SA_model)

#    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
#        param.data = param.data.to(torch.float32)

Parameter containing:
tensor([[-0.0215, -0.0151,  0.0165,  ..., -0.0312,  0.0163, -0.0182],
        [-0.0237, -0.0077,  0.0143,  ...,  0.0146,  0.0277, -0.0028]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0075, -0.0154], device='cuda:0', requires_grad=True)
trainable params: 1538 || all params: 109483778 || trainable%: 0.0014047743219091325


In [11]:
#a = tokenizer(["yes, it is good","asfaf"])
#a
#SA_model(a["input_ids"])

In [12]:
all_data = open("data.csv", "r",encoding="utf-8",errors="ignore").read().split("\n")

texts, labels, max_length = [], [], []
for data in all_data:
    if data:
        line = data.split(",")
        text = line[-1]
        label = line[0]
        max_length.append(len(text))
        texts.append(text)
        labels.append(label)
print(len(texts),len(labels),len(max_length))

1600000 1600000 1600000


In [13]:
#texts[0]
#tokenizer(texts[0],truncation=True,padding="max_length",max_length=512)

In [14]:
from torch.utils.data import Dataset, DataLoader
class TweetDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        output = tokenizer(texts[idx],truncation=True,padding="max_length",max_length=512)
        label = int(labels[idx].strip('"'))
        if label == 4:
            label = 1
        return (torch.tensor(output["input_ids"]), torch.tensor(output["attention_mask"])), torch.tensor(label)

In [15]:
dataset = TweetDataset(texts,labels)
data_size = len(dataset)
#dataset[0]

In [16]:
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(data_size*0.6), int(data_size*0.2), int(data_size*0.2)])
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [17]:
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([para for para in SA_model.parameters() if para.requires_grad==True], lr= 0.01)
acc_max = 0
from sklearn.metrics import accuracy_score
import time
start = time.time()
for epoch in range(epochs):
    loss_sum, count = 0, 0
    print("Epoch {}:".format(epoch))
    SA_model.train()
    for batch_index, (batch_text, batch_label) in enumerate(tqdm(train_dataloader)):
        batch_label = batch_label.to(device)
        pred = SA_model(batch_text)

        loss = loss_fn(pred, batch_label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss
        count += 1
        if count >= 1000:
            print("Loss: {}".format(loss_sum/count))
            loss_sum, count = 0.0, 0
        #if len(train_dataloader) - batch_index <= len(train_dataloader) % 1000 and count == len(train_dataloader) % 1000:
        #    msg = "[{0}/{1:5d}]\tTrain_Loss:{2:.4f}"
        #    print(msg.format(epoch + 1, batch_index + 1, loss_sum / count))
        #    loss_sum, count = 0.0, 0
#
        #if batch_index % 1000 == 999:
        #    msg = "[{0}/{1:5d}]\tTrain_Loss:{2:.4f}"
        #    print(msg.format(epoch + 1, batch_index + 1, loss_sum / count))
        #    loss_sum, count = 0.0, 0

    SA_model.eval()
    all_pred, all_true = [], []
    with torch.no_grad():
        for batch_text, batch_label in tqdm(valid_dataloader):
            batch_label = batch_label.to(device)
            pred = SA_model(batch_text)

            pred = torch.argmax(pred, dim=1).cpu().numpy().tolist()
            label = batch_label.cpu().numpy().tolist()

            all_pred.extend(pred)
            all_true.extend(label)

    acc = accuracy_score(all_pred, all_true)
    print(f"dev acc:{acc:.4f}")
    wait = 0
    if acc > acc_max:
        print(acc, acc_max)
        acc_max = acc
        torch.save(SA_model.state_dict(), "saved_dict/epoch{}_acc{}.pt".format(epoch,acc))
        print(f"Best Model Saved")
    else:
        wait += 1
        if wait >= 3:
            break

torch.save(SA_model.state_dict(), "saved_dict/final.pt")
costtime = time.time() - start


Epoch 0:


  0%|          | 48/120000 [00:08<5:50:21,  5.71it/s]


KeyboardInterrupt: 

In [ ]:
torch.__version__

In [ ]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
len(train_dataloader)

In [ ]:
all_pred, all_true = [], []
from sklearn.metrics import accuracy_score
SA_model.eval()
with torch.no_grad():
    for batch_text, batch_label in tqdm(test_dataloader):
        #print(batch_text, batch_label)
        batch_label = batch_label.to(device)
        pred = SA_model(batch_text)
        pred = torch.argmax(pred, dim=1)

        pred = pred.cpu().numpy().tolist()
        label = batch_label.cpu().numpy().tolist()

        all_pred.extend(pred)
        all_true.extend(label)

accuracy = accuracy_score(all_true, all_pred)

print(f"test dataset accuracy:{accuracy:.4f}")